# 1. Các thư viện cần thiêt

In [25]:
import shutil
import getpass
import time
import datetime
import os
import pandas as pd
import sqlalchemy
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.edge.service import Service
from datetime import datetime, timedelta
import xlwings as xw
from sqlalchemy import create_engine, inspect, text

import warnings
warnings.filterwarnings('ignore')

current_user = getpass.getuser()

In [26]:
# Refresh file Excel API
path_file_api = os.path.join(os.getcwd(),"Crawling_Pipelines.xlsm")
app = xw.App(visible=False)
wb = xw.Book(path_file_api)
wb.api.RefreshAll()
wb.save()
app.kill()
del app

In [27]:
class renamer():
    def __init__(self):
        self.d = dict()

    def __call__(self, x):
        if x not in self.d:
            self.d[x] = 0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])
        
def remove_all_file(move_path,source_file):
    move_path_file = os.path.join(move_path,source_file)
    if os.path.isfile(move_path_file):
        os.remove(move_path_file)

def sql_col_type(df):    
    dtypedict = {}
    for i,j in zip(df.columns, df.dtypes):
        if "object" in str(j):
            dtypedict.update({i: sqlalchemy.types.NVARCHAR(length = 4000)})
                                
        if "datetime" in str(j):
            dtypedict.update({i: sqlalchemy.types.DateTime()})

        if "float" in str(j):
            dtypedict.update({i: sqlalchemy.types.FLOAT(precision=3, asdecimal=True)})

        if "int" in str(j):
            dtypedict.update({i: sqlalchemy.types.INT()})
    return dtypedict


def transform_to_sql(df,database_name,schema_name,table_name,dict_types):
    server = 'VNSGNBIDC002'
    engine = create_engine('mssql+pyodbc://' + server + '/' +database_name + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
    conn = engine.connect()
    start_time = datetime.now()
    df.to_sql(
        table_name,
        con=conn,
        if_exists='replace',
        index=False,
        dtype= dict_types,
        schema = schema_name
    )
    end_time = datetime.now()

    # df_etl = pd.DataFrame(
    #     {
    #         'StartTime':[str(start_time)],
    #         'EndTime':[str(end_time)],
    #         'ObjectName':[table_name],
    #         'RecordRow':[len(df)],
    #         'UserName':[getpass.getuser()]
    #     }
    # )
    # df_etl.to_sql(
    #     table_name,
    #     con = conn,
    #     if_exists='append',
    #     index=False,
    #     schema='EBAO'
    # )
    # conn.execute(f"INSERT INTO DR_Report.EBAO.Table_Log (StartTime, EndTime, ObjectName, RecordRow, UserName) VALUES ('{start_time.strftime('%Y-%m-%d %H:%M:%S')}', '{end_time.strftime('%Y-%m-%d %H:%M:%S')}',  '{table_name}',{len(df)}, '{getpass.getuser()}')")
    conn.commit()
    conn.close()

In [28]:
module = pd.read_excel(path_file_api,sheet_name="Homepage")
login_item_list = pd.read_excel(path_file_api,sheet_name="Login")
module_item_list = pd.read_excel(path_file_api,sheet_name="Module")
import_item_list = pd.read_excel(path_file_api,sheet_name="Import")
# hpage = module[module['Module'] == 'Agent Info']['Page']

In [31]:
lst_module_no = module["Module_No"].unique().tolist()
lst_module_no = [6]

In [32]:
for index_module in lst_module_no:
    hpage = module[module['Module_No'] == index_module]
    login_item = login_item_list[login_item_list['Module_No'] == index_module]
    module_item = module_item_list[module_item_list['Module_No'] == index_module]
    import_item = import_item_list[import_item_list['Module_No'] == index_module]


    for index, row in import_item.iterrows():
        # download_path = row['Download_Path']
        source_file = row['Source_File']
        skip_rows = row['Skip_Row']
        database_name = row['Database']
        schema_name = row['Des_Schema']
        table_name = row['Des_Table']
        file_quantity = row['Files']



    move_path = os.path.join(os.getcwd(),'Task_ETL\\'+hpage['Module'].values[0])

    # Create Target Path
    if os.path.exists(move_path) == False:
        os.makedirs(move_path)

    # Remove file in Downloaded File and Moved File
    lst_file = [os.path.join(move_path,file) for file in os.listdir(move_path)]
    for file in lst_file:
        if os.path.exists(file):
            os.remove(file)
    
    move_path_file = os.path.join(move_path,source_file)


    
    # Create a new Edge browser instance
    options = webdriver.EdgeOptions()
    options.add_experimental_option(
        "prefs", 
        {
            "download.default_directory": move_path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True,
            'profile.default_content_setting_values.automatic_downloads': 1
        }
    )
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    s = Service('C:\Selenium\msedgedriver.exe')
    browser = webdriver.Edge(options=options,service=s)

    link = hpage['Page'].values[0]
    # Navigate to the login page
    browser.get(link)
    browser.implicitly_wait(1)



    # Login Page
    for index, row in login_item.iterrows():
        if row['Element_Type'] == 'NAME' and row['Action'] == 'send_keys':
            item = browser.find_element(By.NAME, row['Element_Value'])
        
            item.send_keys(row['Action_Value'])
        if row['Element_Type'] == 'NAME' and row['Action'] == 'click':
            item = browser.find_element(By.NAME, row['Element_Value'])
            item.click()
        if row['Element_Type'] == 'ID' and row['Action'] == 'click':
            item = browser.find_element(By.ID, row['Element_Value'])
            item.click()
                    
    # Module action
    for index, row in module_item.iterrows():
        if row['Element_Type'] == 'NAME' and row['Action'] == 'send_keys':
            item = browser.find_element(By.NAME, row['Element_Value'])
            item.send_keys(Keys.CONTROL,"a")
            item.send_keys(Keys.DELETE)
            item.send_keys(row['Action_Value'])
        elif (row['Element_Type'] == 'NAME' or row['Element_Type'] == 'ID') and row['Action'] == 'click':
            item = browser.find_element(By.NAME, row['Element_Value'])
            item.click()
            time.sleep(row['Action_Value_Number'])
        elif row['Element_Type'] == 'Script' and row['Action'] == 'run':
            # Case Download Multiple File Then Handle Rename
            if file_quantity > 1:
                for i in range(1,file_quantity + 1):
                    item = row['Element_Value'].replace("(i)",str(i))
                    browser.execute_script(item)
                    rename_path_file = os.path.join(move_path,re.sub('.xls',f'{i}.xls',source_file))
                    browser.implicitly_wait(5)
                    # time.sleep(10)
                    time.sleep(row['Action_Value_Number'])

            else:
                item = row['Element_Value'].replace("i","")
                browser.execute_script(item)
        elif row['Action'] == 'navigation':
            browser.get(row['Action_Value'])
        elif row['Action'] == 'wait':
            browser.implicitly_wait(row['Action_Value_Number'])
    

    # Import To SQL SERVER
    time.sleep(5)
    browser.quit()
    lst_file = [os.path.join(move_path,file) for file in os.listdir(move_path)]
    df_end = pd.DataFrame()
    if len(lst_file) > 1:
        for file in lst_file:
            df = pd.read_excel(file)
            df.columns = df.iloc[skip_rows - 1,:]
            data = df[skip_rows:]
            df_end = pd.concat([df_end,data],axis = 0 )
        df_end.rename(columns = renamer(),inplace=True)
        df_end = df_end.reset_index(drop = True)
        
    else:
        file = os.path.join(move_path,lst_file[0])
        df = pd.read_excel(file)
        df.columns = df.iloc[skip_rows - 1,:]
        data = df[skip_rows:]
        data.rename(columns = renamer(),inplace=True)
        df_end = data.reset_index(drop = True)
    dict_types = sql_col_type(df_end)
    transform_to_sql(df_end,database_name,schema_name,table_name,dict_types)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="m_start_date_hidden"]"}
  (Session info: MicrosoftEdge=115.0.1901.183)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF7E8EA5432+69586]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7E8E35672+760242]
	(No symbol) [0x00007FF7E8BEB8CC]
	(No symbol) [0x00007FF7E8C32C79]
	(No symbol) [0x00007FF7E8C32D8A]
	(No symbol) [0x00007FF7E8C6F007]
	(No symbol) [0x00007FF7E8C52DEF]
	(No symbol) [0x00007FF7E8C26A11]
	(No symbol) [0x00007FF7E8C6C241]
	(No symbol) [0x00007FF7E8C52B83]
	(No symbol) [0x00007FF7E8C2589E]
	(No symbol) [0x00007FF7E8C24B6B]
	(No symbol) [0x00007FF7E8C26024]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF7E907D239+1294553]
	(No symbol) [0x00007FF7E8CA49F4]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7E8D83DF1+33073]
	Microsoft::Applications::Events::EventProperty::~